# Seed Analysis Tutorial

This is a full workflow that shows methods for counting and analyzing the shape and color of seeds. Similar methods should work for other types of seeds.

In [ ]:
# Install PlantCV and required dependencies
%pip install "altair>=5" ipympl plantcv

# Give access and mount your Google Drive (need a Google Account)
# Change path to directory you wish output files to be saved to.
from google.colab import drive
drive.mount('/content/gdrive')

# Enable widget feature with matplotlib
from google.colab import output
output.enable_custom_widget_manager()

#View working directory, may need to change path
%pwd

In [ ]:
# Set the notebook display method
# If widget is not working, then change to inline
%matplotlib widget

# Section 1: Importing Image and Libraries

In [ ]:
# Import libraries
from plantcv import plantcv as pcv
from plantcv.parallel import WorkflowInputs

# Print out the version of PlantCV being used by the Jupyter kernel
pcv.__version__

## Input/Output variables

The options class mimics the workflow command-line argument parser that is used for workflow parallelization. Using it while developing a workflow in Jupyter makes it easier to convert the workflow to a script later.

In [ ]:
# Input image into self.image (include file path if image is not in 
# the same folder as jupyter notebook)

# Set self.debug to "plot" so that image outputs for each step is shown
# once cell is run in jupyter notebooks (recommended)

args = WorkflowInputs(
    images=["./img/beans_img.jpg"],
    names="image1",
    result="seed_analysis_results.json",
    outdir=".",
    writeimg=False,
    debug="plot")

In [ ]:
# Set debug to the global parameter
pcv.params.debug = args.debug

# Set plotting size (default = 100)
pcv.params.dpi = 100

# Increase text size and thickness to make labels clearer
# (size may need to be altered based on original image size)
pcv.params.text_size = 10
pcv.params.text_thickness = 20

## Read the input image

### Google Colaboratory users:
If you are running this notebook using Google Colaboratory, you will need to use the cell below to complete the tutorial unless you cloned the repository from GitHub to your personal Google Drive account. Once you begin developing your workflow for your data, you can delete this text cell and the code cell below so you avoid any conflicts the next time you use this notebook.

In [ ]:
img = pcv.io.open_url(url="https://raw.githubusercontent.com/danforthcenter/plantcv-tutorial-seeds/main/img/beans_img.jpg")

### Local system use:
If you have downloaded this notebook or the repository to your local device, then you will need to use the cell below to bring your image into the notebook. Do not run this line if you are navigating through the **tutorial** using Google Colaboratory.

If you are developing your own workflow using this notebook, make sure you remove the code cell above with the URL and uncomment (remove the hashtag [#] to make the code active. This will call the image you defined during the Inputs/Outputs step.

### Reading images into your environment using *pcv.readimage()*
Inputs:
   * filename = Image file to be read in
   * mode     = How the image will be read into the notebook; either 'native' (default), 'rgb', 'gray', 'csv', or 'envi'

In [ ]:
# img, path, filename = pcv.readimage(filename=args.image1)

# Section 2: Segmenting plant from background and identifying plant object(s)

* Requires successful import of image
* See Threshold Tools Tutorial tutorial for other functions that can be used to create a binary mask

## Color correction

Corrects the color of the input image based on the target color matrix using an affine transformation in the RGB space after automatic detection of a color card within the image.

In [ ]:
corrected_rgb = pcv.transform.auto_correct_color(rgb_img=img)

## Crop image

Cropping out aspects of the image that may interfere with the binary mask makes it easier to isolate plant material from background. This is also useful to save memory in these tutorials. 

In [ ]:
# Inputs:
#   img = color corrected image to get cropped
#   x = top left x-coordinate
#   y = top left y-coordinate
#   h = height of final cropped image
#   w = width of final cropped image
crop_img = pcv.crop(img=corrected_rgb, x=250, y=1200, h=2600, w=2100)

## Visualize colorspaces

The visualization tool converts the color image into HSV and LAB colorspaces and displays the grayscale channels in a matrix so that they can be visualized simultaneously. The idea is to select a channel that maximizes the difference between the plant and the background pixels.

In [ ]:
# Inputs:
#   rbg_img      = cropped image
colorspace_img = pcv.visualize.colorspaces(rgb_img=crop_img)

## Convert the color image to grayscale

Converts the input color image into the LAB colorspace and returns the L (lightness) channel as a grayscale image.

In [ ]:
# Inputs:
#   rbg_img - cropped image 
#   channel - desired colorspace ('l', 'a', or 'b')
b_img = pcv.rgb2gray_lab(rgb_img=crop_img, channel='l')

#Channel L is used because it provides the best separation of seeds from background.

## Visualize the distribution of grayscale values

A histogram can be used to visualize the distribution of values in an image. The histogram can aid in the selection of a threshold value.

For this image, the large peak between 160-220 are from the brighter background pixels. The smaller peak between 60-120 are the darker seed pixels.

In [ ]:
# Inputs:
#   img         = gray image in selected colorspace
#   mask        = None (default), or mask
#   bins        = 100 (default) or number of desired number of evenly spaced bins
#   lower-bound = None (default) or minimum value on x-axis
#   upper-bound = None (default) or maximum value on x-axis
#   title       = None (default) or custom plot title
#   hist_data   = False (default) or True (if frequency distribution data is desired)
hist = pcv.visualize.histogram(img=b_img)

## Threshold the grayscale image
Use a threshold function (binary in this case) to segment the grayscale image into plant (white) and background (black) pixels. Using the histogram above, a threshold point between 130-150 will segment the plant and background peaks. Because the seeds are the lighter pixels in this image, use object_type="light" to do a traditional threshold.

In [ ]:
# Inputs:
#   gray_img    = black and white image created from selected colorspace
#   threshold   = cutoff pixel intensity value (all pixels below value will become black, all above will become white)
#   object_type = 'dark' or 'light' depending on if seeds are darker or lighter than background.
b_thresh = pcv.threshold.binary(gray_img=b_img, threshold=120, object_type='dark') 

## Remove small background noise

Thresholding mostly labeled plant pixels white but also labeled small regions of the background white. The fill function removes "salt" noise from the background by filtering white regions by size.

In [ ]:
# Inputs:
#   bin_img - binary mask image
#   size - maximum size for objects that should be filled in as background (non-plant) pixels
b_fill_salt = pcv.fill(bin_img=b_thresh, size=500)

## Fill holes within objects

Thresholding labeled almost all plant pixels white but left some holes. This function cleans "pepper" noise (black holes) by doing a flood fill.

In [ ]:
# Inputs:
#   bin_img - binary mask image
b_fill = pcv.fill_holes(bin_img=b_fill_salt)
#                                         

# Section 3: Count and Analyze Seeds

* Need a completed binary mask

## Count seeds

Count the number of seeds (simple objects) by accessing the number of values stores in the second item of the object list.

## Measure each seed

1. Create labeled mask
2. Analyze seed shape
3. Analyze seed color

In [ ]:
#Labeled mask is the mask which labels each individual seed to allow it to be measured.

        #Inputs:
        #   Mask = mask created earlier in workflow

labeled_mask, num = pcv.create_labels(mask=b_fill)

In [ ]:
#Find number of objects in image. Num=number of seeds captured in image.

print (num)

In [ ]:
# Analyze shape and size perameters of each seed

        # Inputs:
        #   img = rgb image
        #   labeled_mask = Labeled mask of objects (32-bit).
        #   n_labels = Total number expected individual objects (default = 1).
        #   label = Optional label parameter, modifies the variable name of
        #           observations recorded (default = pcv.params.sample_label)
shape_img = pcv.analyze.size(img=crop_img, labeled_mask=labeled_mask, n_labels=num, label="seed")

In [ ]:
# Analyze color of each seed
        
        # Inputs:
        #   img = rgb image
        #   labeled_mask = Labeled mask of objects (32-bit).
        #   hist_plot_type = 'hsv', or None for no histogram plot
        #   n_labels = Total number expected individual objects (default = 1).
        #   label = Optional label parameter, modifies the variable name of
        #           observations recorded (default = pcv.params.sample_label)
analysis_image = pcv.analyze.color(rgb_img=crop_img, labeled_mask=labeled_mask, n_labels=num, colorspaces="hsv", label="seed")

## Save results

During analysis, measurements are stored in the background in the outputs recorder.

This example includes image analysis for 'area', 'convex_hull_area', 'solidity', 'perimeter', 'width', 'height', 'longest_path', 'center_of_mass, 'convex_hull_vertices', 'object_in_frame', 'ellipse_center', 'ellipse_major_axis', 'ellipse_minor_axis', 'ellipse_angle', 'ellipse_eccentricity' using anayze_object and color analysis using analyze_color.

Here, results are saved to JSON file, and this allows for database concatenation while running workflows in parallel.

In [ ]:
# Inputs:
#   filename  = filename for saving results
#   outformat = output file format: "json" (default) hierarchical format or "csv" tabular format
pcv.outputs.save_results(filename=args.result, outformat="json")